In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 401.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=5
✅ 시뮬레이션 초기화 완료: S=5, Randomness='high', M1=723.00, M2=1049.00


In [2]:
only = gp.Model("only")
only.setParam("OutputFlag", 0)
only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = only.addVars(I, T+1, S, vtype=GRB.CONTINUOUS, name="z")
zc = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = only.addVars(I, T, S, vtype=GRB.BINARY, name="delta")
rho = only.addVars(I, T, S, vtype=GRB.BINARY, name="rho")

only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i, t in product(range(I), range(T))) \
    + gp.quicksum(1 / S * (P_RT[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i, t, s in product(range(I), range(T), range(S)))

only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    only.addConstr(yp[i, t, s] <= R[i, t, s])
    only.addConstr(z[i, t + 1, s] == z[i, t, s] + zc[i, t, s] - zd[i, t, s])
    only.addConstr(zd[i, t, s] <= z[i, t, s])
    only.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    only.addConstr(yp[i, t, s] <= M1 * rho[i, t, s])
    only.addConstr(ym[i, t, s] <= M1 * (1 - rho[i, t, s]))
    only.addConstr(ym[i, t, s] <= M1 * delta[i, t, s])
    only.addConstr(zc[i, t, s] <= M1 * (1 - delta[i, t, s]))
    only.addConstr(zc[i, t, s] <= M1 * zeta[i, t, s])
    only.addConstr(zd[i, t, s] <= M1 * (1 - zeta[i, t, s]))
for i, s in product(range(I), range(S)):
    only.addConstr(z[i, 0, s] == K0[i])

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

# x_vals = np.array([[x[i, t].X for t in range(T)] for i in range(I)])
# yp_vals = np.array([[[yp[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
# ym_vals = np.array([[[ym[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
# z_vals  = np.array([[[z[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
# zc_vals = np.array([[[zc[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
# zd_vals = np.array([[[zd[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])

# x_sum = np.sum(x_vals, axis=0)
# yp_sum = np.sum(yp_vals, axis=0)
# ym_sum = np.sum(ym_vals, axis=0)
# z_sum = np.sum(z_vals, axis=0)
# zc_sum = np.sum(zc_vals, axis=0)
# zd_sum = np.sum(zd_vals, axis=0)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-09
Optimal solution found! Objective value: 848138.7765801009


In [3]:
agg = gp.Model("agg")
agg.setParam("OutputFlag", 0)
agg.setParam("MIPGap", 1e-7)

a = agg.addVars(T, vtype=GRB.CONTINUOUS, name="alpha")
bp = agg.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_plus")
bm = agg.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_minus")
g = agg.addVars(T+1, S, vtype=GRB.CONTINUOUS, name="gamma")
gc = agg.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = agg.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
eta = agg.addVars(T, S, vtype=GRB.BINARY, name="eta")
lam = agg.addVars(T, S, vtype=GRB.BINARY, name="lambda")
mu = agg.addVars(T, S, vtype=GRB.BINARY, name="mu")

agg.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + \
      gp.quicksum(1 / S * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s])
                  for t, s in product(range(T), range(S)))

agg.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s] + gc[t, s] - gd[t, s])
    agg.addConstr(bp[t, s] <= gp.quicksum(R[i, t, s] for i in range(I)))
    agg.addConstr(g[t + 1, s] == g[t, s] + gc[t, s] - gd[t, s])
    agg.addConstr(gd[t, s] <= g[t, s])
    agg.addConstr(gc[t, s] <= sum(K) - g[t, s])
    agg.addConstr(bp[t, s] <= M2 * mu[t, s])
    agg.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
    agg.addConstr(bm[t, s] <= M2 * eta[t, s])
    agg.addConstr(gc[t, s] <= M2 * (1 - eta[t, s]))
    agg.addConstr(gc[t, s] <= M2 * lam[t, s])
    agg.addConstr(gd[t, s] <= M2 * (1 - lam[t, s]))
for s in range(S):
    agg.addConstr(g[0, s] == sum(K0))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")

agg_a_vals  = np.array([agg.getVarByName(f"alpha[{t}]").X for t in range(T)])
agg_bp_vals = np.array([[agg.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
agg_bm_vals = np.array([[agg.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])
agg_gc_vals = np.array([[agg.getVarByName(f"gamma_charge[{t},{s}]").X for s in range(S)] for t in range(T)])
agg_gd_vals = np.array([[agg.getVarByName(f"gamma_discharge[{t},{s}]").X for s in range(S)] for t in range(T)])
agg_g_vals  = np.array([[agg.getVarByName(f"gamma[{t},{s}]").X for s in range(S)] for t in range(T + 1)])

Optimal solution found! Objective value: 858073.8325482183


In [15]:
set = gp.Model("set")
set.setParam("MIPGap", 1e-7)

a = set.addVars(T, vtype=GRB.CONTINUOUS, name="alpha")
bp = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_plus")
bm = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="beta_minus")
g = set.addVars(T + 1, S, vtype=GRB.CONTINUOUS, name="gamma")
gc = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
eta = set.addVars(T, S, vtype=GRB.BINARY, name="eta")
lam = set.addVars(T, S, vtype=GRB.BINARY, name="lambda")
mu = set.addVars(T, S, vtype=GRB.BINARY, name="mu")

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = set.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = set.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = set.addVars(I, T, S, vtype=GRB.BINARY, name="delta")
rho = set.addVars(I, T, S, vtype=GRB.BINARY, name="rho")

dyp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dyp+")
dym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dym+")

dzc_p = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc+")
dzc_m = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzc-")
dzd_p = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd+")
dzd_m = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="dzd-")

q1 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q1")
q2 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q2")
q3 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q3")
q4 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q4")
q5 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q5")
q6 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q6")
q7 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q7")
q8 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q8")
q9 = set.addVars(I, T, S, vtype=GRB.BINARY, name="q9")

e = set.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="e")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")
ec = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_charge")
ed = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_discharge")

set.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(
    (1 / S) * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s])
    for t in range(T)
    for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    set.addConstr(
        gp.quicksum(R[i, t, s] for i in range(I)) - a[t]
        == bp[t, s] - bm[t, s] + gc[t, s] - gd[t, s]
    )
    set.addConstr(bp[t, s] <= gp.quicksum(R[i, t, s] for i in range(I)))
    set.addConstr(g[t + 1, s] == g[t, s] + gc[t, s] - gd[t, s])
    set.addConstr(gd[t, s] <= g[t, s])
    set.addConstr(gc[t, s] <= sum(K) - g[t, s])
    set.addConstr(bp[t, s] <= M2 * mu[t, s])
    set.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
    set.addConstr(bm[t, s] <= M2 * eta[t, s])
    set.addConstr(gc[t, s] <= M2 * (1 - eta[t, s]))
    set.addConstr(gc[t, s] <= M2 * lam[t, s])
    set.addConstr(gd[t, s] <= M2 * (1 - lam[t, s]))
for s in range(S):
    set.addConstr(g[0, s] == sum(K0))

for t in range(T):
    set.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
for t, s in product(range(T), range(S)):
    set.addConstr(bp[t, s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    set.addConstr(bm[t, s] == gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(g[t, s] == gp.quicksum(z[i, t, s] for i in range(I)))
    set.addConstr(gc[t, s] == gp.quicksum(ec[i, t, s] for i in range(I)))
    set.addConstr(gd[t, s] == gp.quicksum(ed[i, t, s] for i in range(I)))
    set.addConstr(g[t + 1, s] == g[t, s] + gc[t, s] - gd[t, s])

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(zd[i, t, s] <= z[i, t, s])
    set.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    set.addConstr(yp[i, t, s] <= M1 * rho[i, t, s])
    set.addConstr(ym[i, t, s] <= M1 * (1 - rho[i, t, s]))
    set.addConstr(ym[i, t, s] <= M1 * delta[i, t, s])
    set.addConstr(zc[i, t, s] <= M1 * (1 - delta[i, t, s]))
    set.addConstr(zc[i, t, s] <= M1 * zeta[i, t, s])
    set.addConstr(zd[i, t, s] <= M1 * (1 - zeta[i, t, s]))
    set.addConstr(z[i, t + 1, s] == z[i, t, s] + ec[i, t, s] - ed[i, t, s])
    set.addConstr(z[i, t, s] <= K[i])
for i, s in product(range(I), range(S)):
    set.addConstr(z[i, 0, s] == K0[i])

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(yp[i, t, s] - ep[i, t, s] == dyp[i, t, s])
    set.addConstr(ym[i, t, s] - em[i, t, s] == dym[i, t, s])
    
    set.addConstr(zc[i, t, s] - ec[i, t, s] == dzc_p[i, t, s] - dzc_m[i, t, s])
    set.addConstr(dzc_p[i, t, s] <= M1 * q3[i, t, s])
    set.addConstr(dzc_m[i, t, s] <= M1 * (1 - q3[i, t, s]))
    set.addConstr(ec[i, t, s] <= K[i] - z[i, t, s])

    set.addConstr(zd[i, t, s] - ed[i, t, s] == dzd_p[i, t, s] - dzd_m[i, t, s])
    set.addConstr(dzd_p[i, t, s] <= M1 * q4[i, t, s])
    set.addConstr(dzd_m[i, t, s] <= M1 * (1 - q4[i, t, s]))
    set.addConstr(ed[i, t, s] <= z[i, t, s])

    set.addConstr(ep[i, t, s] <= M1 * q5[i, t, s])
    set.addConstr(em[i, t, s] <= M1 * (1 - q5[i, t, s]))
    
    set.addConstr(em[i, t, s] <= M1 * q6[i, t, s])
    set.addConstr(ec[i, t, s] <= M1 * (1 - q6[i, t, s]))
    
    set.addConstr(ec[i, t, s] <= M1 * q7[i, t, s])
    set.addConstr(ed[i, t, s] <= M1 * (1 - q7[i, t, s]))

set.optimize()

if set.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {set.objVal}")
else:
    print("No optimal solution found.")

Set parameter MIPGap to value 1e-07
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  1e-07

Optimize a model with 12164 rows, 12611 columns and 29516 nonzeros
Model fingerprint: 0xf55da530
Variable types: 7931 continuous, 4680 integer (4680 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [9e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2190 rows and 4933 columns
Presolve time: 0.03s
Presolved: 9974 rows, 7678 columns, 25232 nonzeros
Variable types: 4765 continuous, 2913 integer (2913 binary)
Found heuristic solution: objective 697648.15252

Root relaxation: objective 8.606900e+05, 2861 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objectiv

In [16]:
a_vals = np.array([set.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_vals = np.array(
    [[set.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
bm_vals = np.array(
    [[set.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)]
)
gc_vals = np.array(
    [[set.getVarByName(f"gamma_charge[{t},{s}]").X for s in range(S)] for t in range(T)]
)
gd_vals = np.array(
    [
        [set.getVarByName(f"gamma_discharge[{t},{s}]").X for s in range(S)]
        for t in range(T)
    ]
)
g_vals = np.array(
    [[set.getVarByName(f"gamma[{t},{s}]").X for s in range(S)] for t in range(T + 1)]
)
x_vals = np.array(
    [[[set.getVarByName(f"x[{i},{t}]").X for s in range(S)] for t in range(T)] for i in range(I)]
)
yp_vals = np.array(
    [
        [
            [set.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ym_vals = np.array(
    [
        [
            [set.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zc_vals = np.array(
    [
        [
            [set.getVarByName(f"z_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
zd_vals = np.array(
    [
        [
            [set.getVarByName(f"z_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
z_vals = np.array(
    [
        [
            [set.getVarByName(f"z[{i},{t},{s}]").X for s in range(S)]
            for t in range(T + 1)
        ]
        for i in range(I)
    ]
)
e_vals = np.array(
    [
        [[set.getVarByName(f"e[{i},{t},{s}]").X for s in range(S)] for t in range(T)]
        for i in range(I)
    ]
)
ep_vals = np.array(
    [
        [
            [set.getVarByName(f"e_plus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
em_vals = np.array(
    [
        [
            [set.getVarByName(f"e_minus[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ec_vals = np.array(
    [
        [
            [set.getVarByName(f"e_charge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
ed_vals = np.array(
    [
        [
            [set.getVarByName(f"e_discharge[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dyp_p_vals = np.array(
    [
        [
            [set.getVarByName(f"dyp+[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
# dyp_m_vals = np.array(
#     [
#         [
#             [set.getVarByName(f"dyp-[{i},{t},{s}]").X for s in range(S)]
#             for t in range(T)
#         ]
#         for i in range(I)
#     ]
# )
dym_p_vals = np.array(
    [
        [
            [set.getVarByName(f"dym+[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
# dym_m_vals = np.array(
#     [
#         [
#             [set.getVarByName(f"dym-[{i},{t},{s}]").X for s in range(S)]
#             for t in range(T)
#         ]
#         for i in range(I)
#     ]
# )
dzc_p_vals = np.array(
    [
        [
            [set.getVarByName(f"dzc+[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dzc_m_vals = np.array(
    [
        [
            [set.getVarByName(f"dzc-[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dzd_p_vals = np.array(
    [
        [
            [set.getVarByName(f"dzd+[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)
dzd_m_vals = np.array(
    [
        [
            [set.getVarByName(f"dzd-[{i},{t},{s}]").X for s in range(S)]
            for t in range(T)
        ]
        for i in range(I)
    ]
)

DA Profit      = 272108.34
RT Profit      = 586676.44
Penalty Cost   = 710.95
Total Profit   = 858073.83, Objective Val  = 858073.83


DA Profit      = 272108.34
RT Profit      = 586676.44
Penalty Cost   = 710.95
Total Profit   = 858073.83, Objective Val  = 858073.83


In [17]:
plot_summary(agg, K, P_DA, P_RT, P_PN, agg_a_vals, agg_bp_vals, agg_bm_vals, agg_g_vals, s=1)
plot_summary(set, K, P_DA, P_RT, P_PN, a_vals, bp_vals, bm_vals, g_vals, s=1)

In [7]:
for s, t, i in product(range(S), range(T), range(I)):
    print(f"[i={i} t={t:02}, s={s:02} BEFORE]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"y⁺={yp_vals[i, t, s]:.2f}, y⁻={ym_vals[i, t, s]:.2f}, "
          f"zᶜ={zc_vals[i, t, s]:.2f}, zᴰ={zd_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")
    
    print(f"[i={i} t={t:02}, s={s:02} SET(-)]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"d⁺={dyp_p_vals[i, t, s]:.2f}, d⁻={dym_p_vals[i, t, s]:.2f}, "
          f"dᶜ={dzc_p_vals[i, t, s]:.2f}, dᴰ={dzd_p_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")
    
    print(f"[i={i} t={t:02}, s={s:02} SET(+)]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"d⁺= -X-, d⁻= -X-,",
          f"dᶜ={dzc_m_vals[i, t, s]:.2f}, dᴰ={dzd_m_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")

    print(f"[i={i} t={t:02}, s={s:02} AFTER ]  "
          f"R={R[i, t, s]:.2f}, x={x_vals[i, t, s]:.2f}, "
          f"e⁺={ep_vals[i, t, s]:.2f}, e⁻={em_vals[i, t, s]:.2f}, "
          f"eᶜ={ec_vals[i, t, s]:.2f}, eᴰ={ed_vals[i, t, s]:.2f}, "
          f"z={z_vals[i, t, s]:.2f}")
    print()

[i=0 t=00, s=00 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=0 t=00, s=00 SET(-)]  R=0.00, x=0.00, d⁺=0.00, d⁻=0.00, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=0 t=00, s=00 SET(+)]  R=0.00, x=0.00, d⁺= -X-, d⁻= -X-, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=0 t=00, s=00 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=1 t=00, s=00 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=1 t=00, s=00 SET(-)]  R=0.00, x=0.00, d⁺=0.00, d⁻=0.00, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=1 t=00, s=00 SET(+)]  R=0.00, x=0.00, d⁺= -X-, d⁻= -X-, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=1 t=00, s=00 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0.00, eᴰ=0.00, z=10.00

[i=2 t=00, s=00 BEFORE]  R=0.00, x=0.00, y⁺=0.00, y⁻=0.00, zᶜ=0.00, zᴰ=0.00, z=10.00
[i=2 t=00, s=00 SET(-)]  R=0.00, x=0.00, d⁺=0.00, d⁻=0.00, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=2 t=00, s=00 SET(+)]  R=0.00, x=0.00, d⁺= -X-, d⁻= -X-, dᶜ=0.00, dᴰ=0.00, z=10.00
[i=2 t=00, s=00 AFTER ]  R=0.00, x=0.00, e⁺=0.00, e⁻=0.00, eᶜ=0

In [8]:
# rows = []
# for t in range(6, 23):
#     for s in range(S):
#         for i in range(I):
#             rows.append({
#                 "i": i,
#                 "t": t,
#                 "s": s,
#                 "공급가능": dyp_p[i, t, s].X + dyp_p[i, t, s].X + dzc_p[i, t, s].X + dzd_p[i, t, s].X,
#                 "수요": dyp_m[i, t, s].X + dym_m[i, t, s].X + dzc_m[i, t, s].X + dzd_m[i, t, s].X,
#                 "yp - ep": yp[i, t, s].X - ep[i, t, s].X,
#                 "dyp+": dyp_p[i, t, s].X,
#                 "dyp-": dyp_m[i, t, s].X,
#                 "ym - em": ym[i, t, s].X - em[i, t, s].X,
#                 "dym+": dym_p[i, t, s].X,
#                 "dym-": dym_m[i, t, s].X,
#                 "zc - ec": zc[i, t, s].X - ec[i, t, s].X,
#                 "dzc+": dzc_p[i, t, s].X,
#                 "dzc-": dzc_m[i, t, s].X,
#                 "zd - ed": zd[i, t, s].X - ed[i, t, s].X,
#                 "dzd+": dzd_p[i, t, s].X,
#                 "dzd-": dzd_m[i, t, s].X,
#             })

# pd.DataFrame(rows)

NameError: name 'dyp_m' is not defined